In [1]:
import torch
import syft as sy
import pandas as pd
from syft.lib.python.list import List
print(sy.__version__)
print(torch.__version__)

0.5.0
1.8.1+cu102


### Make the VM and its client

In [2]:
server = sy.VirtualMachine(name="server")
root_client = server.get_root_client()
client = server.get_client()

In [3]:
print(type(server))
print(server)
print(type(root_client))
print(root_client)
print(client)

<class 'syft.core.node.vm.vm.VirtualMachine'>
VirtualMachine: server: <UID: e7c69f319e1c4f2ca9e0a1df00d91c12>
<class 'syft.core.node.vm.client.VirtualMachineClient'>
<VirtualMachineClient: server Client>
<VirtualMachineClient: server Client>


### Check out the remote modules

In [4]:
remote_pd = root_client.pandas
remote_torch = root_client.torch
remote_np = root_client.numpy
remote_vision = root_client.torchvision

In [5]:
print(type(remote_pd))
remote_pd

<class 'syft.ast.module.Module'>


Module:
	.DataFrame -> <syft.ast.klass.Class object at 0x7f3c3e52b520>
	.Series -> <syft.ast.klass.Class object at 0x7f3c3e52b580>
	.CategoricalDtype -> <syft.ast.klass.Class object at 0x7f3c3e52b5e0>
	.Categorical -> <syft.ast.klass.Class object at 0x7f3c3e52b640>
	.json_normalize -> <syft.ast.callable.Callable object at 0x7f3c3e52b700>

In [7]:
remote_np

Module:
	.ndarray -> <syft.ast.klass.Class object at 0x7fbbf58185e0>

In [8]:
remote_vision

Module:
	.__version__ -> <syft.ast.static_attr.StaticAttribute object at 0x7fbbf593ce80>
	.transforms -> Module:
		.Compose -> <syft.ast.klass.Class object at 0x7fbbf593cf40>
		.ToTensor -> <syft.ast.klass.Class object at 0x7fbbf593cfa0>
		.Normalize -> <syft.ast.klass.Class object at 0x7fbbf57f00a0>
		.CenterCrop -> <syft.ast.klass.Class object at 0x7fbbf57f9460>
		.ColorJitter -> <syft.ast.klass.Class object at 0x7fbbf57f9520>
		.FiveCrop -> <syft.ast.klass.Class object at 0x7fbbf57f9580>
		.Grayscale -> <syft.ast.klass.Class object at 0x7fbbf57f9640>
		.Pad -> <syft.ast.klass.Class object at 0x7fbbf57f96a0>
		.RandomAffine -> <syft.ast.klass.Class object at 0x7fbbf57f9760>
		.RandomApply -> <syft.ast.klass.Class object at 0x7fbbf57f9820>
		.RandomCrop -> <syft.ast.klass.Class object at 0x7fbbf57f98e0>
		.RandomGrayscale -> <syft.ast.klass.Class object at 0x7fbbf57f99a0>
		.RandomHorizontalFlip -> <syft.ast.klass.Class object at 0x7fbbf57f9a60>
		.RandomPerspective -> <syft.ast.klass

In [9]:
remote_torch

Module:
	.Tensor -> <syft.ast.klass.Class object at 0x7fbbf5acb9a0>
	.BFloat16Tensor -> <syft.ast.klass.Class object at 0x7fbbf5acba00>
	.BoolTensor -> <syft.ast.klass.Class object at 0x7fbbf5acba60>
	.ByteTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbac0>
	.CharTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbb20>
	.DoubleTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbb80>
	.FloatTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbbe0>
	.HalfTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbc40>
	.IntTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbca0>
	.LongTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbd00>
	.ShortTensor -> <syft.ast.klass.Class object at 0x7fbbf5acbd60>
	.nn -> Module:
		.Parameter -> <syft.ast.klass.Class object at 0x7fbbf5acbe80>
		.Module -> <syft.ast.klass.Class object at 0x7fbbf5a1d340>
		.Conv2d -> <syft.ast.klass.Class object at 0x7fbbf5a1d820>
		.Dropout2d -> <syft.ast.klass.Class object at 0x7fbbf5a1dc40>
		.Linear -> <syft.

### Sending server's data to the client

In [6]:
x = torch.tensor([1,2,3]).tag("hey", "there", "buddy")
x.description = "wow what a day"
x_ptr = x.send(root_client)

In [7]:
root_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 3ac15d40eb884c2dad99b90e8b805006>,"[hey, there, buddy]",wow what a day,<class 'torch.Tensor'>


In [9]:
client.store.pandas  # should not show anything

,ID,Tags,Description,object_type
0,<UID: 3ac15d40eb884c2dad99b90e8b805006>,"[hey, there, buddy]",wow what a day,<class 'torch.Tensor'>


### Sending client's data to the server

In [13]:
root_client_data_ptr = remote_torch.randn(size=(1, 100))
print(f'pointer to client data: {root_client_data_ptr}')
client_data = root_client_data_ptr.get()
print(client_data)
print(client_data.device)

pointer to client data: <syft.proxy.torch.TensorPointer object at 0x7f3c3d9bf220>
tensor([[-0.8490, -0.3528,  1.1262,  2.5993, -0.0522,  2.0475,  0.2407,  0.5821,
         -1.3793,  1.1260,  0.1360,  0.2554,  1.4984,  1.1111,  0.1778, -0.2173,
          0.5765,  0.3791,  0.8984, -0.8942,  1.4765,  1.6995,  2.1662,  0.7103,
          1.0295,  2.5990, -1.1429, -0.1411, -0.6498, -0.5277,  0.6104,  0.1412,
          0.4084, -0.5618,  0.6036, -0.3896,  0.5497,  1.7065,  0.0769,  0.1981,
         -0.3803,  0.2298,  0.2763,  0.2191,  0.3057,  1.1697, -0.9724, -0.2015,
         -0.7625,  0.6670, -1.9188,  0.4393, -0.2626, -0.2723,  0.2192,  0.5234,
          1.4330, -1.9719, -1.5744,  1.1942,  0.3785, -1.0146, -0.7447,  0.3490,
          1.0636, -0.8486,  0.5488,  0.8967, -0.5624, -0.6214,  0.4557,  0.8365,
         -0.8060,  0.1352,  0.4598, -1.0482, -0.2478, -0.4842,  0.8842, -0.9080,
          1.9151, -0.8233,  2.0673,  1.3958, -1.6099,  1.4482, -0.4422, -0.2822,
          0.8713, -1.2737, 

### Checking out client's device (cuda or CPU)

For root client

In [7]:
root_client_cuda_ptr = remote_torch.cuda.is_available()
root_client_cuda_ptr.id_at_location

<UID: 3655ddc2605444e4916d80b9401026e1>

In [8]:
bool_cuda = root_client_cuda_ptr.get_copy()
print(f'root client has CUDA? {bool_cuda}')

cuda_ptr = remote_torch.device("cuda" if bool_cuda else "cpu")
print(f'root client device: {cuda_ptr.type.get_copy()}')

root client has CUDA? True
root client device: cuda


What if the server requests a cuda tensor?

In [17]:
root_client_data_ptr = remote_torch.randn(size=(1, 100)).cuda()
print(f'pointer to client data: {root_client_data_ptr}')
try:
    client_data = root_client_data_ptr.get()
    print(client_data)
    print(client_data.device)
except TypeError as e:
    print(e)

pointer to client data: <syft.proxy.torch.TensorPointer object at 0x7f3c3d9f7640>
can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.


Check out device for normal client (somehow syft asks for request, which I don't know how to accept yet)

In [20]:
from syft.core.node.common.service.auth import AuthorizationException
remote_torch2 = client.torch
bool_cuda_ptr2 = remote_torch2.cuda.is_available()
try:
    bool_cuda2 = bool_cuda_ptr2.get_copy()
    print(f'client has CUDA? {bool_cuda2}')
    cuda_ptr = remote_torch2.device("cuda" if bool_cuda else "cpu")
    print(f'client device: {cuda_ptr.type.get_copy()}')
except AuthorizationException as e:
    print(e)

[2021-09-10T07:13:23.135178+0000][CRITICAL][logger]][4012] You do not have permission to .get() Object with ID: <UID: 2a52d046239644e4abfd6a0a90154e31>Please submit a request.
[2021-09-10T07:13:23.135717+0000][CRITICAL][logger]][4012] You do not have permission to .get() Object with ID: <UID: 2a52d046239644e4abfd6a0a90154e31>Please submit a request.
[2021-09-10T07:13:23.137206+0000][CRITICAL][logger]][4012] You do not have permission to .get() Object with ID: <UID: 2a52d046239644e4abfd6a0a90154e31>Please submit a request.


You do not have permission to .get() Object with ID: <UID: 2a52d046239644e4abfd6a0a90154e31>Please submit a request.


### Sending data and do operations

In [10]:
t1 = torch.tensor([1, 2])
t2 = torch.tensor([1, 3])
syft_list = List([t1, t2])
syft_list_ptr = syft_list.send(client)

In [11]:
res = syft_list_ptr.get_copy()
res

[tensor([1, 2]), tensor([1, 3])]

In [12]:
tensor_type='uint8'
op_name = '__add__'

In [13]:
x = torch.tensor([-1, 0, 1, 2, 3, 4])
xp = x.send(client)
op_method = getattr(xp, op_name, None)
target_op_method = getattr(x, op_name)

print(op_method)
print(target_op_method)

<bound method get_run_class_method.<locals>.run_class_method of <syft.proxy.torch.TensorPointer object at 0x7f92d2bad640>>
<built-in method __add__ of Tensor object at 0x7f92d2b40700>


In [14]:
result = op_method(xp)  # op(xp, xp)
target_result = target_op_method(x)

local_result = result.get()

assert (local_result == target_result).all()

In [15]:
local_result

tensor([-2,  0,  2,  4,  6,  8])